<h1>Import libraries</h1>

In [ ]:
# Internal Library to register model
!pip install fosforml
from fosforml import *

In [ ]:
pip install pyspark

<h1>Establish connection to snowflake</h1>

In [ ]:
#estblishing connection between notebook and snowflake
import numpy as np
import pandas as pd
from tqdm import tqdm

import time
from snowflake.snowpark.session import Session
import configparser

import warnings
warnings.filterwarnings("ignore")

config = configparser.ConfigParser()
config.read("snowflake_connection.ini")

connection_parameters = {
    "user": f'{config["Snowflake"]["user"]}',
    "password": f'{config["Snowflake"]["password"]}',
    "account": f'{config["Snowflake"]["account"]}',
    "WAREHOUSE": f'{config["Snowflake"]["WAREHOUSE"]}',
    "DATABASE": f'{config["Snowflake"]["DATABASE"]}',
    "SCHEMA": f'{config["Snowflake"]["SCHEMA"]}'
}

def snowflake_connector(conn):
    try:
        session = Session.builder.configs(conn).create()
        print("connection successful!")
    except:
        raise ValueError("error while connecting with db")
    return session

session = snowflake_connector(connection_parameters)

<h1>Analyze data</h1>

In [ ]:
sales_bangalore_2022 = session.table("SOL_ASSORTMENT_PLANNING_COMB").to_pandas()

In [ ]:
sales_bangalore_2022.head()

In [ ]:
train_final = session.table("SOL_ASSORTMENT_PLANNING_TRAIN_DATA_MODEL").to_pandas()
train_final.head(1)

In [ ]:
train_final.columns = train_final.columns.str.lower()

In [ ]:
train_final.head(1)

In [ ]:
master_data = sales_bangalore_2022.copy()
master_data.head(1)

In [ ]:
master_data.columns = master_data.columns.str.lower()
master_data.head(1)

In [ ]:
df1=master_data.copy
master_data['unique_id']=master_data['product_code']+'-'+ master_data['outlet_code']

<h1>Define train test split</h1>

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(train_final[['frequencym','sales_value_avg','scheme_amount_perproduct']],train_final['sales_indicator'],test_size=0.3,random_state=42)

In [ ]:
import pickle

with open('logreg.pkl', 'rb') as model_file:
   loaded_model = pickle.load(model_file)
probs = loaded_model.predict_proba(X_test)
probs

In [ ]:
cutoff = 0.3
y_pred = (probs[:,1]>=cutoff).astype(int)
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

In [ ]:
test_final = session.table("SOL_ASSORTMENT_PLANNING_TEST_SET_MODEL").to_pandas()
test_final.columns = test_final.columns.str.lower()
test_final.head(1)


In [ ]:
X_test_unseen = test_final[['frequencym','sales_value_avg','scheme_amount_perproduct']]
y_test_unseen = test_final['sales_indicator']
print(X_test_unseen.shape,y_test_unseen.shape)
probs = loaded_model.predict_proba(X_test_unseen)
test_final['probablity']=probs[:,1]
y_pred = (probs[:,1]>=0.4).astype(int)
test_final

In [ ]:
test_final['prediction']= test_final['probablity'].apply(lambda x: 1  if x>=0.3 else 0)
test_final

In [ ]:
test_final_static = test_final.copy()
print(classification_report(test_final['sales_indicator'],test_final['prediction']))

In [ ]:
test_final.reset_index(drop=True)
test_final['OOS'] = test_final['probablity'].apply(lambda x:1 if x>0.3 else 0)

In [ ]:
soq_master = master_data.groupby('unique_id').agg({'sales_units':'sum','mnth_code':'nunique'})
soq_master['SOQ'] = soq_master['sales_units']/soq_master['mnth_code']
soq_master.reset_index()
final_reco = pd.merge(soq_master,test_final,on='unique_id',how='inner')

In [ ]:
final_reco = final_reco[['unique_id','prediction','SOQ','OOS']]
final_reco.head()

In [ ]:
ms_reco = final_reco.copy()
ms_reco.columns = ['unique_id', 'ms_flag', 'soq', 'oos_flag']

In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [ ]:
ms_reco[['product', 'outlet']] = ms_reco['unique_id'].str.split('-', expand=True)
ms_reco.head()

In [ ]:
ms_reco=session.createDataFrame(
        ms_reco.values.tolist(),
        schema=ms_reco.columns.tolist())

In [ ]:
#Writing back to Snowflake
ms_reco.write.mode("overwrite").save_as_table("SOL_CPG_DB.SOL_SALES_SCHEMA.SOL_ASSORTMENT_PREDICTION_TABLE")

In [ ]:
req = requests.Request()
req.json = {"payload":X_test_unseen.head(1).to_json()}
print({'payload': req.json})

In [ ]:
from fosforml import *

@scoring_func
def score(model, request):
    payload = request.json["payload"]
    data = pd.DataFrame(eval(payload))
    prediction = pd.DataFrame(model.predict(data))
    return prediction.to_json()

In [ ]:
print(score(loaded_model, req))

In [ ]:
from fosforml.constants import MLModelFlavours


In [ ]:
tmp = register_model(loaded_model, 
               score, 
               name="Assortment_Planning_Prediction", 
               description="prediction of assortment across retailers by product",
               flavour=MLModelFlavours.sklearn,
               model_type="regression",
               y_true=y_test,
               y_pred=y_pred, 
               prob=probs, 
               features=X_train.columns,
               labels=[0,1],
               input_type="json", 
               explain_ai=True, 
               x_train=X_train, 
               x_test=X_test, 
               y_train=y_train.tolist(),
               y_test=y_test.tolist(),
               feature_names=X_train.columns.tolist(),
               original_features=X_train.columns.tolist(),
               feature_ids=X_train.columns,
               target_names=['No Failure',' or Failure'],
               kyd=True, kyd_score = True)